In [7]:
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

In [ ]:
data = pd.read_csv("flux_tower_15days.csv", header = 0, low_memory = False)
skiprows = [0, 2, 3],
print(data.shape)
data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'], format = '%Y-%m-%d %H:%M:%S')
print(data.dtypes)
data.set_index('TIMESTAMP', inplace = True, drop = False)
print(type(data.index))
print(data)

In [ ]:
day_mask = (data.index.time >= datetime.strptime('08:30', '%H:%M').time()) & \
           (data.index.time <= datetime.strptime('17:30', '%H:%M').time())

night_mask = (data.index.time >= datetime.strptime('21:30', '%H:%M').time()) | \
           (data.index.time <= datetime.strptime('05:30', '%H:%M').time())

day_data = data[day_mask]
night_data = data[night_mask]

In [ ]:
def parse_datetime(timestamp):
    try:
        return pd.to_datetime(timestamp, format = '%Y-%m-%d %H:%M:%S')
    except ValueError:
        return pd.to_datetime(timestamp, format = '%Y-%m-%d %H:%M:%S.%f')
data['TIMESTAMP'] = data['TIMESTAMP'].apply(parse_datetime)
print(type(data.index))

In [ ]:
def plotting(data, title_suffix, start_timestamp, end_timestamp):
    data['10min'] = data.index.floor('10T')

    grouped_data_u = data.groupby('10min').mean().rename(columns={'Ux': 'Ux_avg'})
    aligned_avg_data_u = grouped_data_u.reindex(data.index, method='ffill')

    grouped_data_v = data.groupby('10min').mean().rename(columns={'Uy': 'Uy_avg'})
    aligned_avg_data_v = grouped_data_v.reindex(data.index, method='ffill')

    grouped_data_w = data.groupby('10min').mean().rename(columns={'Uz': 'Uz_avg'})
    aligned_avg_data_w = grouped_data_w.reindex(data.index, method='ffill')

    # Calculate Mean Kinetic Energy (MKE)
    MKE = 0.5 * (aligned_avg_data_u['Ux_avg']**2 + aligned_avg_data_v['Uy_avg']**2 + aligned_avg_data_w['Uz_avg']**2)

    # Calculate fluctuations
    fluctuation_u = data['Ux'] - aligned_avg_data_u['Ux_avg']
    fluctuation_v = data['Uy'] - aligned_avg_data_v['Uy_avg']
    fluctuation_w = data['Uz'] - aligned_avg_data_w['Uz_avg']

    # Calculate Turbulent Kinetic Energy (TKE)
    TKE = 0.5 * (fluctuation_u**2 + fluctuation_v**2 + fluctuation_w**2)

    # Calculate turbulence intensity (TKE/MKE)
    turbulence_intensity = TKE / MKE

    # Filter data based on start and end timestamps
    filtered_data = data[(data['TIMESTAMP'] >= start_timestamp) & (data['TIMESTAMP'] <= end_timestamp)]

    # Plotting
    fig_U = px.line(x=filtered_data['TIMESTAMP'], y=fluctuation_u, title=f'U Fluctuation {title_suffix}')
    fig_U.show()

    fig_V = px.line(x=filtered_data['TIMESTAMP'], y=fluctuation_v, title=f'V Fluctuation {title_suffix}')
    fig_V.show()

    fig_W = px.line(x=filtered_data['TIMESTAMP'], y=fluctuation_w, title=f'W Fluctuation {title_suffix}')
    fig_W.show()

    # Plot turbulence intensity
    turbulence_intensity_filtered = turbulence_intensity[(turbulence_intensity.index >= start_timestamp) & (turbulence_intensity.index <= end_timestamp)]
    fig_turbulence_intensity = px.line(x=turbulence_intensity_filtered.index, y=turbulence_intensity_filtered, title=f'Turbulence Intensity {title_suffix}')
    fig_turbulence_intensity.show()

In [ ]:
start = pd.to_datetime('9 june 2015 21:30:00')
end = pd.to_datetime('10 june 2015 5:30:00')

filtered_data = night_data[(night_data.index >= start) & (night_data.index <= end)]

plotting(filtered_data, 'night', start, end)

In [ ]:
def calculate_wind_direction(Ux_avg, Uy_avg):
    angle = np.arctan2(Uy_avg, Ux_avg)  # Calculate the angle in radians
    angle_deg = np.degrees(angle)  # Convert the angle to degrees
    angle_deg = (angle_deg + 360) % 360  # Ensure the angle is between 0 and 360
    return angle_deg

# Function to calculate wind direction for a given time range
def wind_direction_for_time_range(data, start_time, end_time):
    data['10min'] = data.index.floor('10T')
    
    # Group by 10-minute intervals and calculate mean values
    grouped_data = data.groupby('10min').mean()
    
    # Calculate wind direction using 10-minute mean values
    wind_direction = calculate_wind_direction(grouped_data['Ux'], grouped_data['Uy'])
    
    # Filter wind direction data based on start and end time
    wind_direction_filtered = wind_direction[(wind_direction.index >= start_time) & (wind_direction.index <= end_time)]
    
    return wind_direction_filtered

# Define the time range from 9th June to 12th June
start_time = pd.to_datetime('10 June 2015 21:30:00')
end_time = pd.to_datetime('11 June 2015 5:30:00')

# Calculate wind direction for the specified time range
wind_direction = wind_direction_for_time_range(data, start_time, end_time)

# Plot the wind direction against time
fig_wind_dir = px.scatter(x=wind_direction.index, y=wind_direction, title='Wind Direction', labels={'x': 'Time', 'y': 'Wind Direction (degrees)'})

fig_wind_dir.update_layout(
    title={'font': {'size': 24}},
    xaxis={'title': {'font': {'size': 20}}},
    yaxis={'title': {'font': {'size': 20}}},
    font={'size': 16}
)

fig_wind_dir.show()

In [ ]:
def calculate_mke_tke(grouped_data_u, grouped_data_v, grouped_data_w, data):
    # Calculate Mean Kinetic Energy (MKE)
    MKE = 0.5 * (grouped_data_u**2 + grouped_data_v**2)
    
    # Calculate fluctuations
    fluctuation_u = data['Ux'] - grouped_data_u
    fluctuation_v = data['Uy'] - grouped_data_v
    fluctuation_w = data['Uz'] - grouped_data_w

    # Calculate Turbulent Kinetic Energy (TKE)
    TKE = 0.5 * (fluctuation_u**2 + fluctuation_v**2)
    
    # Align TKE with the 10-minute intervals
    TKE_avg = TKE.groupby(data['10min']).mean()

    return MKE, TKE_avg

In [ ]:
# Function to calculate wind direction and turbulence intensity
def wind_direction_and_turbulence_intensity(data, start_time, end_time):
    data['10min'] = data.index.floor('10T')
    
    # Group by 10-minute intervals and calculate mean values
    grouped_data = data.groupby('10min').mean()
    
    # Calculate wind direction using 10-minute mean values
    wind_direction = calculate_wind_direction(grouped_data['Ux'], grouped_data['Uy'])
    
    # Calculate MKE and TKE
    MKE, TKE = calculate_mke_tke(grouped_data['Ux'], grouped_data['Uy'], grouped_data['Uz'], data)
    
    # Calculate turbulence intensity
    turbulence_intensity = TKE / MKE

    # Filter data based on start and end time
    wind_direction_filtered = wind_direction[(wind_direction.index >= start_time) & (wind_direction.index <= end_time)]
    turbulence_intensity_filtered = turbulence_intensity[(turbulence_intensity.index >= start_time) & (turbulence_intensity.index <= end_time)]
    TKE_filtered = TKE[(TKE.index >= start_time) & (TKE.index <= end_time)]
    
    return wind_direction_filtered, turbulence_intensity_filtered_filtered

In [ ]:
def wind_direction_and_tke(data, start_time, end_time):
    data['10min'] = data.index.floor('10T')
    
    # Group by 10-minute intervals and calculate mean values
    grouped_data = data.groupby('10min').mean()
    
    # Calculate wind direction using 10-minute mean values
    wind_direction = calculate_wind_direction(grouped_data['Ux'], grouped_data['Uy'])
    
    # Calculate MKE and TKE
    MKE, TKE = calculate_mke_tke(grouped_data['Ux'], grouped_data['Uy'], grouped_data['Uz'], data)
    
    # Calculate turbulence intensity
    turbulence_intensity = TKE / MKE

    # Filter data based on start and end time
    wind_direction_filtered = wind_direction[(wind_direction.index >= start_time) & (wind_direction.index <= end_time)]
    turbulence_intensity_filtered = turbulence_intensity[(turbulence_intensity.index >= start_time) & (turbulence_intensity.index <= end_time)]
    TKE_filtered = TKE[(TKE.index >= start_time) & (TKE.index <= end_time)]
    
    return wind_direction_filtered, TKE_filtered

wind_direction, TKE = wind_direction_and_tke(data, start_time, end_time)

# Plot turbulence intensity against wind direction
fig_tke_vs_wind_dir = px.scatter(
    x=wind_direction, 
    y=turbulence_intensity, 
    title='Turbulent Kinetic Energy vs Wind Direction for 9th-10th June Night', 
    labels={'x': 'Wind Direction (degrees)', 'y': 'Turbulence Intensity (TKE/MKE)'}
)



# Update layout to increase font size
fig_tke_vs_wind_dir.update_layout(
    title={'font': {'size': 24}},
    xaxis={'title': {'font': {'size': 20}}},
    yaxis={'title': {'font': {'size': 20}}},
    font={'size': 16}
)

fig_tke_vs_wind_dir.show()

In [ ]:
def plot_turbulence_vs_wind_direction(data, start_date, end_date):
    wind_direction_all = pd.Series(dtype=float)
    turbulence_intensity_all = pd.Series(dtype=float)
    
    current_date = start_date
    while current_date < end_date:
        start_time = pd.to_datetime(f'{current_date} 21:30:00')
        end_time = start_time + timedelta(hours = 9)
        
        wind_direction, tke = wind_direction_and_turbulence_intensity(data, start_time, end_time)
        
        # Append to the overall series
        wind_direction_all = pd.concat([wind_direction_all, wind_direction])
        turbulence_intensity_all = pd.concat([turbulence_intensity_all, turbulence_intensity])
        
        # Move to the next day
        current_date += timedelta(days=1)
    
    # Plot turbulence intensity against wind direction
    fig_turbulence_vs_wind_dir = px.scatter(
        x=wind_direction_all, 
        y=turbulence_intensity_all, 
        title='Turbulence Intensity vs Wind Direction for Multiple Nights', 
        labels={'x': 'Wind Direction (degrees)', 'y': 'Turbulence Intensity (TKE/MKE)'}
    )

    # Update layout to increase font size
    fig_turbulence_vs_wind_dir.update_layout(
        title={'font': {'size': 24}},
        xaxis={'title': {'font': {'size': 20}}},
        yaxis={'title': {'font': {'size': 20}}},
        font={'size': 16}
    )

    fig_turbulence_vs_wind_dir.show()

# Define the time range from 9th June to 12th June
start_date = pd.to_datetime('1 June 2015')
end_date = pd.to_datetime('15 June 2015')

# Plot turbulence intensity vs wind direction for multiple nights
plot_turbulence_vs_wind_direction(data, start_date, end_date)